# Deliverable 2. Create a Customer Travel Destinations Map

----

1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Geoapify API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found".

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame. In the point for each city add:

    * The city name
    
    * The country code
    
    * The weather description
    
    * The point's size should be the maximum temperature for the city

15. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.

---

## Ensure that the dependencies and the Geoapify API key is imported correctly

In [42]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import os


# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`

In [2]:
# Import the WeatherPy_database.csv file. 
weather_data=os.path.join("..",".","Weather_Database","WeatherPy_Database.csv")

city_data_df=pd.read_csv(weather_data)
# Display sample data
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Condition
0,0,Carnarvon,AU,-24.8667,113.6333,82.98,51,88,11.39,light rain
1,1,Bredasdorp,ZA,-34.5322,20.0403,66.25,80,73,4.99,broken clouds
2,2,Kruisfontein,ZA,-34.0033,24.7314,73.89,64,2,14.34,clear sky
3,3,Ushuaia,AR,-54.8000,-68.3000,64.06,55,0,5.75,clear sky
4,4,Busselton,AU,-33.6500,115.3333,62.19,73,97,8.68,overcast clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation

In [3]:
# Prompt the user to enter minimum and maximum temperature criteria.
min_temp=float(input("What is the minimum temperature you would like for your vacation?"))
max_temp=float(input("What is the maximum temperature you would like for your vacation?"))

What is the minimum temperature you would like for your vacation?70
What is the maximum temperature you would like for your vacation?90


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`

In [4]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc method.
## remember to put "&" statement in brackets [()()]
preferred_cities_df=city_data_df.loc[(city_data_df["Max Temp"]>= min_temp) 
                                     & (city_data_df["Max Temp"]<=max_temp)]
# Display sample data
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Condition
0,0,Carnarvon,AU,-24.8667,113.6333,82.98,51,88,11.39,light rain
2,2,Kruisfontein,ZA,-34.0033,24.7314,73.89,64,2,14.34,clear sky
7,7,Sao Jose De Piranhas,BR,-7.1206,-38.5019,86.54,42,87,2.95,overcast clouds
17,17,Chuy,UY,-33.6971,-53.4616,74.46,57,4,15.68,clear sky
20,20,Sao Felix Do Xingu,BR,-6.6447,-51.9950,70.21,99,100,1.90,heavy intensity rain


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows

In [5]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.isna().sum()
clean_travel_cities=preferred_cities_df.dropna()

# Display sample data
clean_travel_cities.isna().sum() #check data if empty row has been dropped

clean_travel_cities

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Condition
0,0,Carnarvon,AU,-24.8667,113.6333,82.98,51,88,11.39,light rain
2,2,Kruisfontein,ZA,-34.0033,24.7314,73.89,64,2,14.34,clear sky
7,7,Sao Jose De Piranhas,BR,-7.1206,-38.5019,86.54,42,87,2.95,overcast clouds
17,17,Chuy,UY,-33.6971,-53.4616,74.46,57,4,15.68,clear sky
20,20,Sao Felix Do Xingu,BR,-6.6447,-51.9950,70.21,99,100,1.90,heavy intensity rain
...,...,...,...,...,...,...,...,...,...,...
696,696,Puerto Lempira,HN,15.2667,-83.7667,81.19,70,42,8.55,light rain
698,698,Jaffna,LK,9.6685,80.0074,77.54,78,100,14.99,overcast clouds
699,699,Ibirapitanga,BR,-14.1642,-39.3736,73.76,98,99,2.19,overcast clouds
700,700,Damara,CF,4.9608,18.7035,75.47,28,97,2.89,overcast clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng"

In [6]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
# to copy use .copy() at the end of the varible.

hotel_df=clean_travel_cities[["City","Country","Max Temp","Current Condition","Lat","Lng"]].copy()


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame

In [7]:
# Create a new column "Hotel Name".
#to create an empty column use new travel_cities and have ["Hotel Name"] = []
hotel_df["Hotel Name"] = ""
# Display sample data
hotel_df.head()

,City,Country,Max Temp,Current Condition,Lat,Lng,Hotel Name
0,Carnarvon,AU,82.98,light rain,-24.8667,113.6333,
2,Kruisfontein,ZA,73.89,clear sky,-34.0033,24.7314,
7,Sao Jose De Piranhas,BR,86.54,overcast clouds,-7.1206,-38.5019,
17,Chuy,UY,74.46,clear sky,-33.6971,-53.4616,
20,Sao Felix Do Xingu,BR,70.21,heavy intensity rain,-6.6447,-51.9950,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city

In [13]:
# Set parameters to search for a hotel

radius = 5000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}


## Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found"

In [9]:
# Iterate through the hotel DataFrame

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame.
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set up the base URL for the Geoapify Places API.
    base_url = "https://api.geoapify.com/v2/places"

    # Make request and retrieve the JSON data by using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found set the hotel name as "No hotel found".
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# # Display sample data
hotel_df

Starting hotel search
Carnarvon - nearest hotel: No hotel found
Kruisfontein - nearest hotel: No hotel found
Sao Jose De Piranhas - nearest hotel: Hotel Pousada São José
Chuy - nearest hotel: Alerces
Sao Felix Do Xingu - nearest hotel: Hotel Terraço
Mar Del Plata - nearest hotel: Nuevo Ostende
Jizan - nearest hotel: No hotel found
Souillac - nearest hotel: Hotel SkylineInn
Port Elizabeth - nearest hotel: Waterford Hotel
Port Alfred - nearest hotel: No hotel found
Georgetown - nearest hotel: Page 63 hostel
San Cristobal - nearest hotel: Terra Sur
Tocopilla - nearest hotel: hotel Isidora
Sao Filipe - nearest hotel: Bela Vista
Saint-Philippe - nearest hotel: Le Baril
Upington - nearest hotel: Die Eiland Holiday Ressort
Qandala - nearest hotel: No hotel found
Acarau - nearest hotel: No hotel found
Bambous Virieux - nearest hotel: No hotel found
Wajir - nearest hotel: Ngamia Club
Vila Velha - nearest hotel: Hotel Prainha
Coquimbo - nearest hotel: Hotel Iberia
Hambantota - nearest hotel: Oas

Sechura - nearest hotel: Hospedaje Claritza
Nioro - nearest hotel: No hotel found
Raga - nearest hotel: No hotel found
Kismayo - nearest hotel: Kismayo Hotel
Saint-Georges - nearest hotel: Deyna's City Inn
Amarante Do Maranhao - nearest hotel: No hotel found
Sumbe - nearest hotel: No hotel found
Vera Cruz - nearest hotel: No hotel found
Ambilobe - nearest hotel: Hôtel National
Faranah - nearest hotel: Hotel Selima
Yenagoa - nearest hotel: Peretimi Hotels Ltd.
Kang - nearest hotel: Nkisi Guest House
Itarema - nearest hotel: Pousada Oásis
Salvador - nearest hotel: No hotel found
Padang - nearest hotel: losmen Surya
Treinta Y Tres - nearest hotel: No hotel found
Mogadishu - nearest hotel: SYL Hotel
Nuevitas - nearest hotel: Hotel Caonaba
Tabou - nearest hotel: hôtel le rochet
Saint-Joseph - nearest hotel: No hotel found
Bambey - nearest hotel: No hotel found
Trairi - nearest hotel: No hotel found
Namikupa - nearest hotel: No hotel found
Isla Mujeres - nearest hotel: Villa La Bella
Pimente

,City,Country,Max Temp,Current Condition,Lat,Lng,Hotel Name
0,Carnarvon,AU,82.98,light rain,-24.8667,113.6333,No hotel found
2,Kruisfontein,ZA,73.89,clear sky,-34.0033,24.7314,No hotel found
7,Sao Jose De Piranhas,BR,86.54,overcast clouds,-7.1206,-38.5019,Hotel Pousada São José
17,Chuy,UY,74.46,clear sky,-33.6971,-53.4616,Alerces
20,Sao Felix Do Xingu,BR,70.21,heavy intensity rain,-6.6447,-51.9950,Hotel Terraço
...,...,...,...,...,...,...,...
696,Puerto Lempira,HN,81.19,light rain,15.2667,-83.7667,Hotel Samaritano
698,Jaffna,LK,77.54,overcast clouds,9.6685,80.0074,Three Star Hotel
699,Ibirapitanga,BR,73.76,overcast clouds,-14.1642,-39.3736,No hotel found
700,Damara,CF,75.47,overcast clouds,4.9608,18.7035,No hotel found


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`

In [10]:
#import dependecies , using numpy
import numpy as np
# Drop the rows where there is no Hotel Name.
#use .replace() to replace "No hotel found to NaN" eg. df=df.replace("name", np.nan)
hotel_df = hotel_df.replace("No hotel found", np.nan)
hotel_df.isna().sum()
#drop the Nan
clean_hotel_df=hotel_df.dropna()

# Display sample data
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Condition,Lat,Lng,Hotel Name
7,Sao Jose De Piranhas,BR,86.54,overcast clouds,-7.1206,-38.5019,Hotel Pousada São José
17,Chuy,UY,74.46,clear sky,-33.6971,-53.4616,Alerces
20,Sao Felix Do Xingu,BR,70.21,heavy intensity rain,-6.6447,-51.9950,Hotel Terraço
21,Mar Del Plata,AR,86.83,overcast clouds,-38.0023,-57.5575,Nuevo Ostende
27,Souillac,MU,76.17,overcast clouds,-20.5167,57.5167,Hotel SkylineInn
28,Port Elizabeth,ZA,76.10,clear sky,-33.9180,25.5701,Waterford Hotel
40,Georgetown,MY,79.90,few clouds,5.4112,100.3354,Page 63 hostel
44,San Cristobal,VE,78.98,few clouds,7.7669,-72.2250,Terra Sur
45,Tocopilla,CL,73.17,few clouds,-22.0920,-70.1979,hotel Isidora
46,Sao Filipe,CV,76.86,clear sky,14.8961,-24.4956,Bela Vista


## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the `Vacation_Search`` folder

In [11]:
# Create the output File (CSV)
output_data_file="../Vacation_Search/WeatherPy_vacation.csv"

# Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file,index_label="City_ID")

## Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame.

In the point for each city add:

* The city name
* The country code
* The weather description
* The point's size should be the maximum temperature for the city

In [43]:

# Configure the map plot, using .hvplot.points
city_map=clean_hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles="OSM",
    size="Max Temp",
    color= "City",
    alpha=0.5,
    frame_width = 600,
    frame_height = 500,
    hover_cols = ["Current Condition", "Country"])
# Display the map
city_map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Current Condition,Country)